<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/07_document_classification_20201219.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 07 単語埋め込みを使った文書分類
* 今回も、IMDbデータセットの感情分析を文書分類問題として解く。
* ただし今回は、fastTextのような学習済みの単語埋め込みは使わない。
* 単語埋め込み自体の学習も、ネットワークの学習と同時におこなう。
* IMDbデータの準備も、`torch.torchtext`を使っておこなう。
 * つまりすべてをPyTorchのなかでおこなう。
* 参考資料
 * https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb
 * https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/4%20-%20Convolutional%20Sentiment%20Analysis.ipynb

## データをどう扱うか
* ネットワークへの入力は、単語埋め込みを、単語の出現順どおりに並べた列にする。
 * ミニバッチは[ミニバッチのなかでの最大文書長, ミニバッチのサイズ, 単語埋め込み次元数]という形の3階のテンソルになる。
* そして、前向き計算のなかではじめて、単語埋め込みの平均をとることにする。
 * `.mean(0)`と、軸0で平均をとることになる。


## 07-00 Google Colabのランタイムのタイプを変更する
* Google ColabのランタイムのタイプをGPUに変更しておこう。
 * 上のメニューの「ランタイム」→「ランタイムのタイプを変更」→「ハードウェア　アクセラレータ」から「GPU」を選択

## 07-01 torchtextを使ってIMDbデータを読み込む
* ここでIMDbデータセットの読み込みにつかう`torchtext.datasets`については、下記を参照。
 * https://torchtext.readthedocs.io/en/latest/datasets.html

### 実験の再現性確保のための設定など
* https://pytorch.org/docs/stable/notes/randomness.html

In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG']=":4096:8"


In [2]:
import random
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchtext import data
from torchtext import datasets
from torchtext.data import Field, LabelField, BucketIterator

SEED = 123

random.seed(SEED)
torch.manual_seed(SEED)
torch.set_deterministic(True)

### torchtextのフィールド
* TEXTフィールドと、LABELフィールドという２種類のFieldオブジェクトのインスタンスを作る。
 * Fieldクラスの詳細については[ここ](https://github.com/pytorch/text/blob/master/torchtext/data/field.py)を参照。
* TEXTフィールドは、テキストの前処理の仕方を決めておくのに使う。
 * tokenizerは、デフォルトでは単にstring型のsplitメソッドを適用するだけになる。これは高速だが、tokenizationとしては雑。
* LABELフィールドは、ラベルの前処理に使う。

In [3]:
TEXT = Field(tokenize="spacy")
LABEL = LabelField()

### IMDbデータセットをダウンロードした後、前処理しつつ読み込む
* ダウンロードはすぐ終わるが、解凍に少し時間がかかる。
* また、TEXTフィールドでspaCyのtokenizationを使うように設定したので、少し時間がかかる。
 * string型のsplitメソッドでtokenizeすると、時間はあまりかからない。（そのかわり、やや雑なtokenizationになる。）

In [4]:
train_valid_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

### 最初の文書を見てみる

In [5]:
print(train_valid_data[0].text)

['I', 'disagree', 'strongly', 'with', 'anyone', 'who', 'might', 'dismiss', 'this', 'film', 'as', '"', 'just', '"', 'entertainment', '.', 'Set', 'right', 'after', 'the', 'carefree', ',', 'roaring', '20s', ',', 'during', 'the', 'early', 'days', 'of', 'the', 'Great', 'Depression', ',', 'Dance', ',', 'Fools', ',', 'Dance', 'is', 'at', 'its', 'heart', 'an', 'earnest', 'cautionary', 'tale', ',', 'with', 'a', 'clear', 'message', 'about', 'how', 'best', 'to', 'endure', 'these', 'hard', 'times', '.', 'Yet', 'this', 'fast', '-', 'paced', 'and', 'tightly', '-', 'plotted', 'film', 'is', 'far', 'from', 'being', 'a', 'dreary', 'morality', 'tale.<br', '/><br', '/>In', 'the', '30s', ',', 'Hollywood', 'had', 'a', 'knack', 'for', 'churning', 'out', 'one', 'entertaining', '*', 'and', '*', 'enlightening', 'audience', '-', 'pleaser', 'after', 'another', ',', 'all', 'without', 'wasting', 'a', 'frame', 'of', 'film', '.', 'Dance', ',', 'Fools', ',', 'Dance', '--', 'one', 'of', '*', 'four', '*', 'films', 'that

In [6]:
print(train_valid_data[0].label)

pos


### テストセット以外の部分を訓練データと検証データに分ける

In [7]:
train_data, valid_data = train_valid_data.split(split_ratio=0.8)

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 20000
Number of validation examples: 5000
Number of testing examples: 25000


In [9]:
print(train_data[0].text)

['This', 'is', 'one', 'of', 'the', 'greatest', 'child', '-', 'pet', 'movies', 'ever', 'created', '.', 'I', 'cry', 'every', 'time', 'I', 'see', 'Shadow', 'yelling', '"', 'Wait', ',', 'wait', 'for', 'me', 'Peter', '!', '"', 'as', 'the', 'family', 'car', 'is', 'pulling', 'away', '.', 'This', 'is', 'a', 'must', 'see', 'if', 'you', 'love', 'animals', '!', 'Best', 'Movie', 'Ever', '!', 'The', 'lines', 'in', 'the', 'movie', 'are', 'sometimes', 'stupid', '.', 'Like', 'when', 'Sassy', 'says', 'to', 'Chance', ';', '"', 'Cat', "'s", 'Rule', 'and', 'dogs', 'drool', '!', '"', 'Lines', 'like', 'this', 'I', 'could', 'do', 'without', ',', 'but', 'when', 'I', 'was', 'six', 'I', 'bet', 'I', 'loved', 'that', 'line', '.', 'The', 'storyline', 'may', 'seem', 'hooky', 'to', 'some', ',', 'but', 'I', 'like', 'it', '.', 'Shadow', 'as', 'the', 'older', 'dog', 'who', "'s", 'preparing', 'Chance', 'to', 'take', 'over', 'for', 'him', 'when', 'he', "'s", 'gone', 'is', 'really', 'moving', 'when', 'you', 'think', 'abou

### データセットの語彙とラベルを作る
* TEXTラベルのほうでは、最大語彙サイズを指定する。

In [10]:
MAX_VOCAB_SIZE = 25000 # この値は適当。

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

なぜ語彙サイズが25,000ではなく25,002なのかについては、少し下の説明を参照。

In [11]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


### 出現頻度順で上位２０単語を見てみる

In [12]:
print(TEXT.vocab.freqs.most_common(20))

[('the', 231466), (',', 220572), ('.', 189097), ('a', 125017), ('and', 125012), ('of', 114875), ('to', 106992), ('is', 87340), ('in', 70294), ('I', 61900), ('it', 61164), ('that', 56231), ('"', 50199), ("'s", 49490), ('this', 48385), ('-', 42420), ('/><br', 40924), ('was', 40198), ('as', 34764), ('with', 34223)]


### 単語ID順に最初の１０単語を見てみる
* IDのうち、0と1は、未知語とパディング用の単語という特殊な単語に割り振られている。
 * 未知語は`<unk>`という特殊な単語に置き換えられる。これのIDが0。
 * パディングとは、長さが不揃いの複数の文書を同じミニバッチにまとめるとき、すべての文書の長さを無理やりそろえるため、文書末尾に特殊な単語（元々の語彙にない、人工的に用意した単語）を追加すること。
 * パディング用の単語が`<pad>`になっているのは、上のほうで使ったFieldクラスのインスタンスを作るときのデフォルトの値がこの`<pad>`になっているため。

In [13]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', ',', '.', 'a', 'and', 'of', 'to', 'is']


### ラベルのほうのIDを確認する
* こちらはnegとposに対応する２つのIDしかない。

In [14]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


### ミニバッチを取り出すためのiteratorを作る
* ミニバッチのサイズを指定する。
 * ミニバッチのサイズは、性能を出すためにチューニングすべきハイパーパラメータのひとつ。

In [15]:
BATCH_SIZE = 100

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator = BucketIterator(train_data, batch_size=BATCH_SIZE, device=device,
                                     sort_within_batch=True, shuffle=True, sort_key=lambda x: len(x.text))
valid_iterator = BucketIterator(valid_data, batch_size=BATCH_SIZE, device=device)
test_iterator = BucketIterator(test_data, batch_size=BATCH_SIZE, device=device)

### ミニバッチの中身を確認する

* 訓練データのiteratorを回してミニバッチをすべて取得してみる
 * ミニバッチのshapeを表示してみる

In [16]:
for i, batch in enumerate(train_iterator):
  print(i, batch.text.shape)

0 torch.Size([133, 100])
1 torch.Size([454, 100])
2 torch.Size([248, 100])
3 torch.Size([213, 100])
4 torch.Size([106, 100])
5 torch.Size([374, 100])
6 torch.Size([94, 100])
7 torch.Size([187, 100])
8 torch.Size([286, 100])
9 torch.Size([112, 100])
10 torch.Size([49, 100])
11 torch.Size([531, 100])
12 torch.Size([190, 100])
13 torch.Size([161, 100])
14 torch.Size([227, 100])
15 torch.Size([135, 100])
16 torch.Size([386, 100])
17 torch.Size([829, 100])
18 torch.Size([182, 100])
19 torch.Size([170, 100])
20 torch.Size([243, 100])
21 torch.Size([921, 100])
22 torch.Size([1055, 100])
23 torch.Size([69, 100])
24 torch.Size([300, 100])
25 torch.Size([270, 100])
26 torch.Size([329, 100])
27 torch.Size([216, 100])
28 torch.Size([612, 100])
29 torch.Size([128, 100])
30 torch.Size([275, 100])
31 torch.Size([223, 100])
32 torch.Size([154, 100])
33 torch.Size([472, 100])
34 torch.Size([63, 100])
35 torch.Size([175, 100])
36 torch.Size([398, 100])
37 torch.Size([438, 100])
38 torch.Size([121, 100])

* ミニバッチの形は、[ミニバッチ内での最大文書長, ミニバッチのサイズ]になっていることに注意！
 * ミニバッチのサイズが最初に来ているのではない！
 * [ミニバッチのサイズ, ミニバッチ内での最大文書長]という形にしたいなら、テキストのfieldを作るとき以下のようにする。

__`TEXT = data.Field(tokenize="spacy", batch_first=True)`__

* 上記のループを抜けたあとには、変数batchには最後のミニバッチが代入されている。

In [17]:
batch.text.shape

torch.Size([139, 100])

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を表示させてみる。

In [18]:
print(batch.text[:, 0])

tensor([   11,   216,    14,    25,  4676,     7, 17659,    14,    19,     5,
           62,    59, 17411,   710,    22,     3,    21,    60,  3409,   375,
           45,  1460,  8578,     3,  2819, 24356,     6,  3640,     0,     4,
          173,    16,    78,    43,   426,     7,     2,   225,    29,   239,
            2,  1328,    30,    12,    63,   264,    36,    38,     7,    83,
         1668,   786,    17,   664,   569,     7,     2,  5589,    18, 13953,
            3,    11,   160,   236,  6596, 24356,     8,  1525,     3,   450,
         5012,     0,    85,    27,   952,    49,    20,    95,    23,    90,
            7,     2,   225,     6,    72,    85,    34,     8,  4498,     2,
         7731,   164,     0,   130,     4,   173,    31,   217,    54,     2,
            0,  1328,    52,     2, 12691,     6,     2,    14, 15005,    14,
          175,    10,   172,   158,     3,   123,    16,    22,     9,  1184,
          590,     4,  3401,    83,   924,  1668,     7,     2, 

* このミニバッチに含まれる文書のうち、最初の文書の単語ID列を単語列に戻したものを表示させてみる。

In [19]:
print(' '.join([TEXT.vocab.itos[i] for i in batch.text[:, 0]]))

I thought " The River of Souls " was a very good Babylon 5 movie , with some exceptional performances from Martin Sheen , Tracy Scoggins and Ian <unk> . If this were an episode of the series ( without the humour ) it would probably be one of my favourite stand - alone stories of the series.<br /><br />Personally , I 've always preferred Scoggins to Christian , although granted <unk> did n't write her as well for much of the series and she did have to endure the Byron / <unk> plot . If you take out the <unk> humour about the brothel and the " poorer " actors in those scenes , then this movie is solid stuff . Probably my third favourite of the four movies , but in no means bad at all .


* このミニバッチに含まれる文書のうち、最後の文書の単語ID列を表示させてみる。

In [20]:
print(batch.text[:, BATCH_SIZE-1])

tensor([   11,    34,    91,     5,   356,     7,    16,    22,    23,   176,
            6,   176,     4,  1448,     7, 16091,     0,   875,    93,     2,
        20137,     3,    11,    74,     8,   217,     2,    22,   142,     2,
         4367,     6,   126,    12,     4,   567,   101,   169,    10,  5920,
           85,    33,    81,   112,   425,    16,    22,    19,    10,   818,
           84,    58,  3733,  6274,     6,  1916,  6953,    32,   818,  3004,
            4,   764,    11,   133,    34,   178,  9864,  2183, 11328,     3,
           16,    22,  1146,    84,   209,    68,    11,   126,    12,     4,
           25,   131,     9,   377,     8,   805,    23,     5,   248,     6,
            2,   527,     9,    48,  1475,     4,    11,   648,   398,    13,
           31,   126,    16,    76,    31,    34,    27,   494,     4,   240,
           96,    94,     8,    81,  3165, 12228,     3,  5487,  3889,     3,
        16091, 18484,     3,     6,  4427,  3886,    88,     8, 

最後の文書の末尾は「1」で埋められていることが分かる。

この1は、パディング用単語のIDだったことを想起されたい。

ミニバッチに含まれる文書の長さを調べると、文書が文書長の降順に並べられていることが分かる。

In [21]:
(batch.text != 1).sum(0)

tensor([139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139, 139,
        139, 139, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138, 138,
        138, 138, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137,
        137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137, 137,
        137, 137], device='cuda:0')

## 07-02 MLPによる文書分類の準備
* 今回は、ごく簡単なMLPで文書分類をする。
* 文書中の全単語トークンの埋め込みベクトルの平均を、MLPの入力とする。
 * 当然、語順の情報は使われない。
 * つまり、bag-of-wordsモデルになっている。

### 定数の設定
* 単語埋め込みベクトルの次元数を設定

### 次元を128→512に

In [22]:
INPUT_DIM = len(TEXT.vocab)
NUM_CLASS = len(LABEL.vocab)
EMBED_DIM = 512
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

print(f'語彙サイズ {INPUT_DIM}, クラス数 {NUM_CLASS}, 単語埋め込み次元 {EMBED_DIM}')

語彙サイズ 25002, クラス数 2, 単語埋め込み次元 512


### モデルを定義する前にPyTorchの単語埋め込みがどんなものかを見てみる
* 埋め込みとは、単語IDから単語ベクトルへのマッピング。

* 以下のように、語彙サイズと埋め込みの次元数を指定しつつ、torch.nn.Embeddingのインスタンスを作ればよい。

In [23]:
embed = nn.Embedding(INPUT_DIM, EMBED_DIM, padding_idx=PAD_IDX).to(device)

* パディング用の単語の埋め込みはゼロベクトルになる。

In [24]:
print(embed(torch.tensor([21,1]).to(device)))

tensor([[-1.3173, -0.5923,  0.6092,  ..., -0.9463,  0.4801,  0.3610],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]],
       device='cuda:0', grad_fn=<EmbeddingBackward>)


* 埋め込みの効果を見てみる

In [25]:
for i, batch in enumerate(train_iterator):
  print(i, embed(batch.text).shape)

0 torch.Size([401, 100, 512])
1 torch.Size([143, 100, 512])
2 torch.Size([281, 100, 512])
3 torch.Size([105, 100, 512])
4 torch.Size([207, 100, 512])
5 torch.Size([171, 100, 512])
6 torch.Size([511, 100, 512])
7 torch.Size([88, 100, 512])
8 torch.Size([81, 100, 512])
9 torch.Size([136, 100, 512])
10 torch.Size([230, 100, 512])
11 torch.Size([198, 100, 512])
12 torch.Size([347, 100, 512])
13 torch.Size([211, 100, 512])
14 torch.Size([753, 100, 512])
15 torch.Size([201, 100, 512])
16 torch.Size([921, 100, 512])
17 torch.Size([367, 100, 512])
18 torch.Size([148, 100, 512])
19 torch.Size([301, 100, 512])
20 torch.Size([584, 100, 512])
21 torch.Size([294, 100, 512])
22 torch.Size([195, 100, 512])
23 torch.Size([438, 100, 512])
24 torch.Size([616, 100, 512])
25 torch.Size([222, 100, 512])
26 torch.Size([62, 100, 512])
27 torch.Size([239, 100, 512])
28 torch.Size([829, 100, 512])
29 torch.Size([264, 100, 512])
30 torch.Size([219, 100, 512])
31 torch.Size([182, 100, 512])
32 torch.Size([193, 1

### モデルの定義
* MLP（多層パーセプトロン）だが、入り口に単語埋め込み層が挿入されている。

### 初めの層を1024に

In [26]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx):
    super(EmbedTextSentiment, self).__init__()
    self.padding_idx = padding_idx # 2020/12/19追加
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.fc1 = nn.Linear(embed_dim, 1024)
    self.fc2 = nn.Linear(1024, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.embed(text)
    #x = x.mean(0) # 文書に含まれる全単語トークンの単語ベクトルの平均
    #上の平均の計算を正確に書くと、以下の2行のようになります。（2020/12/19追加）
    mask = (text != self.padding_idx)
    x = (x * mask.unsqueeze(2)).sum(0) / mask.sum(0).unsqueeze(1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

### モデルを作る
* モデル（のインスタンス）をGPUに移動させている点に注意。

In [27]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX).to(device)

### 損失関数とoptimizerとschedulerを作る

### lr を0.001に

In [28]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

### 訓練用の関数
* 最初の`model.train()`に注意。こうやって、モデルを訓練モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [29]:
def train(data_iterator, model, optimizer, scheduler, criterion):

  model.train()

  train_loss = 0
  train_acc = 0
  for batch in data_iterator:
    optimizer.zero_grad()
    text, cls = batch.text, batch.label
    output = model(text)
    loss = criterion(output, cls)
    train_loss += loss.item()
    loss.backward()
    optimizer.step()
    train_acc += (output.argmax(1) == cls).float().mean().item()

  scheduler.step()

  num_batch = len(data_iterator)
  return train_loss / num_batch, train_acc / num_batch

### 評価用の関数
* 最初の`model.eval()`に注意。こうやって、モデルを評価モードに設定する。
 * 例えば、dropoutを含むモデルなど、訓練時と評価時で、ふるまい方を変える必要があるときがあるため、こういうことをする。

In [30]:
def test(data_iterator, model, criterion):

  model.eval()

  loss = 0
  acc = 0
  for batch in data_iterator:
    text, cls = batch.text, batch.label
    with torch.no_grad():
      output = model(text)
      loss = criterion(output, cls)
      loss += loss.item()
      acc += (output.argmax(1) == cls).float().mean().item()

  num_batch = len(data_iterator)
  return loss / num_batch, acc / num_batch

## 07-03 分類器の訓練と検証セットでの評価

### エポック数を100に

In [31]:
N_EPOCHS = 100
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 5 seconds | lr=0.000950
	Loss: 0.48000(train)	|	Acc: 75.82%(train)
	Loss: 0.02058(valid)	|	Acc: 82.66%(valid)
Epoch 2 | time in 0 minutes, 4 seconds | lr=0.000902
	Loss: 0.27400(train)	|	Acc: 89.07%(train)
	Loss: 0.01430(valid)	|	Acc: 86.40%(valid)
Epoch 3 | time in 0 minutes, 4 seconds | lr=0.000857
	Loss: 0.18338(train)	|	Acc: 93.30%(train)
	Loss: 0.01140(valid)	|	Acc: 88.04%(valid)
Epoch 4 | time in 0 minutes, 4 seconds | lr=0.000815
	Loss: 0.12105(train)	|	Acc: 95.86%(train)
	Loss: 0.01135(valid)	|	Acc: 88.18%(valid)
Epoch 5 | time in 0 minutes, 4 seconds | lr=0.000774
	Loss: 0.07044(train)	|	Acc: 97.88%(train)
	Loss: 0.01413(valid)	|	Acc: 88.12%(valid)
Epoch 6 | time in 0 minutes, 4 seconds | lr=0.000735
	Loss: 0.04270(train)	|	Acc: 98.85%(train)
	Loss: 0.03023(valid)	|	Acc: 87.42%(valid)
Epoch 7 | time in 0 minutes, 4 seconds | lr=0.000698
	Loss: 0.02149(train)	|	Acc: 99.57%(train)
	Loss: 0.01842(valid)	|	Acc: 87.30%(valid)
Epoch 8 | time in 0 minutes

Epoch 60 | time in 0 minutes, 4 seconds | lr=0.000046
	Loss: 0.00000(train)	|	Acc: 100.00%(train)
	Loss: 0.05310(valid)	|	Acc: 87.48%(valid)
Epoch 61 | time in 0 minutes, 4 seconds | lr=0.000044
	Loss: 0.00000(train)	|	Acc: 100.00%(train)
	Loss: 0.05827(valid)	|	Acc: 87.46%(valid)
Epoch 62 | time in 0 minutes, 4 seconds | lr=0.000042
	Loss: 0.00000(train)	|	Acc: 100.00%(train)
	Loss: 0.09207(valid)	|	Acc: 87.48%(valid)
Epoch 63 | time in 0 minutes, 4 seconds | lr=0.000039
	Loss: 0.00000(train)	|	Acc: 100.00%(train)
	Loss: 0.06829(valid)	|	Acc: 87.48%(valid)
Epoch 64 | time in 0 minutes, 4 seconds | lr=0.000038
	Loss: 0.00000(train)	|	Acc: 100.00%(train)
	Loss: 0.10514(valid)	|	Acc: 87.48%(valid)
Epoch 65 | time in 0 minutes, 4 seconds | lr=0.000036
	Loss: 0.00000(train)	|	Acc: 100.00%(train)
	Loss: 0.06478(valid)	|	Acc: 87.48%(valid)
Epoch 66 | time in 0 minutes, 5 seconds | lr=0.000034
	Loss: 0.00000(train)	|	Acc: 100.00%(train)
	Loss: 0.06844(valid)	|	Acc: 87.46%(valid)
Epoch 91 | ti

## 07-04 再検討
* 訓練データ上での分類精度がほぼ100%になってしまっている。
* 検証データでの分類精度と大きな差があり、明らかにオーバーフィッティング。

### ドロップアウトを使う
* モデルのインスタンスを作るときにdropoutの確率を引数pで指定できるようにする。

In [41]:
class EmbedTextSentiment(nn.Module):
  def __init__(self, embed_dim, num_class, vocab_size, padding_idx, p=0.0):
    super(EmbedTextSentiment, self).__init__()
    self.padding_idx = padding_idx # 2020/12/19追加
    self.embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
    self.dropout = nn.Dropout(p=p)
    self.fc1 = nn.Linear(embed_dim, 1024)
    self.fc2 = nn.Linear(1024, 100)
    self.fc3 = nn.Linear(100, num_class)

  def forward(self, text):
    x = self.dropout(self.embed(text)) #埋め込み層の直後にdropout
    #x = x.mean(0)
    #上の平均の計算を正確に書くと、以下の2行のようになります。（2020/12/19追加）
    mask = (text != self.padding_idx)
    x = (x * mask.unsqueeze(2)).sum(0) / mask.sum(0).unsqueeze(1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [42]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [ ]:
N_EPOCHS = 100
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 1 | time in 0 minutes, 5 seconds | lr=0.000950
	Loss: 0.53182(train)	|	Acc: 73.20%(train)
	Loss: 0.01476(valid)	|	Acc: 82.70%(valid)
Epoch 2 | time in 0 minutes, 5 seconds | lr=0.000902
	Loss: 0.37805(train)	|	Acc: 83.63%(train)
	Loss: 0.01656(valid)	|	Acc: 86.02%(valid)
Epoch 3 | time in 0 minutes, 5 seconds | lr=0.000857
	Loss: 0.31108(train)	|	Acc: 87.06%(train)
	Loss: 0.01612(valid)	|	Acc: 87.56%(valid)
Epoch 4 | time in 0 minutes, 5 seconds | lr=0.000815
	Loss: 0.26604(train)	|	Acc: 89.12%(train)
	Loss: 0.01526(valid)	|	Acc: 86.14%(valid)
Epoch 5 | time in 0 minutes, 4 seconds | lr=0.000774
	Loss: 0.23360(train)	|	Acc: 90.58%(train)
	Loss: 0.01506(valid)	|	Acc: 88.70%(valid)
Epoch 6 | time in 0 minutes, 5 seconds | lr=0.000735
	Loss: 0.20254(train)	|	Acc: 91.86%(train)
	Loss: 0.00898(valid)	|	Acc: 88.64%(valid)
Epoch 7 | time in 0 minutes, 5 seconds | lr=0.000698
	Loss: 0.18239(train)	|	Acc: 92.96%(train)
	Loss: 0.01073(valid)	|	Acc: 88.76%(valid)
Epoch 8 | time in 0 minutes

Epoch 60 | time in 0 minutes, 5 seconds | lr=0.000046
	Loss: 0.01419(train)	|	Acc: 99.52%(train)
	Loss: 0.01932(valid)	|	Acc: 88.56%(valid)
Epoch 61 | time in 0 minutes, 5 seconds | lr=0.000044
	Loss: 0.01312(train)	|	Acc: 99.49%(train)
	Loss: 0.02708(valid)	|	Acc: 88.32%(valid)
Epoch 62 | time in 0 minutes, 5 seconds | lr=0.000042
	Loss: 0.01384(train)	|	Acc: 99.49%(train)
	Loss: 0.02122(valid)	|	Acc: 88.40%(valid)
Epoch 63 | time in 0 minutes, 5 seconds | lr=0.000039
	Loss: 0.01099(train)	|	Acc: 99.63%(train)
	Loss: 0.02326(valid)	|	Acc: 88.40%(valid)
Epoch 64 | time in 0 minutes, 5 seconds | lr=0.000038
	Loss: 0.01364(train)	|	Acc: 99.58%(train)
	Loss: 0.02699(valid)	|	Acc: 88.40%(valid)
Epoch 65 | time in 0 minutes, 5 seconds | lr=0.000036
	Loss: 0.01311(train)	|	Acc: 99.56%(train)
	Loss: 0.02860(valid)	|	Acc: 88.52%(valid)
Epoch 66 | time in 0 minutes, 5 seconds | lr=0.000034
	Loss: 0.01256(train)	|	Acc: 99.60%(train)
	Loss: 0.02781(valid)	|	Acc: 88.52%(valid)
Epoch 67 | time in 0

### L２正則化を使う
* optimizerのweight_decayパラメータを0より大きな値にする。

In [ ]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [45]:
N_EPOCHS = 100
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

Epoch 21 | time in 0 minutes, 5 seconds | lr=0.000341
	Loss: 0.32830(train)	|	Acc: 86.29%(train)
	Loss: 0.01462(valid)	|	Acc: 86.34%(valid)
Epoch 22 | time in 0 minutes, 5 seconds | lr=0.000324
	Loss: 0.31768(train)	|	Acc: 87.07%(train)
	Loss: 0.01073(valid)	|	Acc: 87.16%(valid)
Epoch 23 | time in 0 minutes, 5 seconds | lr=0.000307
	Loss: 0.31307(train)	|	Acc: 86.94%(train)
	Loss: 0.01918(valid)	|	Acc: 87.48%(valid)
Epoch 24 | time in 0 minutes, 5 seconds | lr=0.000292
	Loss: 0.30524(train)	|	Acc: 87.46%(train)
	Loss: 0.01391(valid)	|	Acc: 87.46%(valid)
Epoch 25 | time in 0 minutes, 5 seconds | lr=0.000277
	Loss: 0.29485(train)	|	Acc: 87.79%(train)
	Loss: 0.01231(valid)	|	Acc: 87.78%(valid)
Epoch 26 | time in 0 minutes, 5 seconds | lr=0.000264
	Loss: 0.28863(train)	|	Acc: 88.39%(train)
	Loss: 0.00853(valid)	|	Acc: 87.82%(valid)
Epoch 27 | time in 0 minutes, 5 seconds | lr=0.000250
	Loss: 0.27876(train)	|	Acc: 88.72%(train)
	Loss: 0.01063(valid)	|	Acc: 87.88%(valid)
Epoch 28 | time in 0

Epoch 80 | time in 0 minutes, 4 seconds | lr=0.000017
	Loss: 0.12422(train)	|	Acc: 95.94%(train)
	Loss: 0.00800(valid)	|	Acc: 88.64%(valid)
Epoch 81 | time in 0 minutes, 5 seconds | lr=0.000016
	Loss: 0.12367(train)	|	Acc: 95.83%(train)
	Loss: 0.01189(valid)	|	Acc: 88.54%(valid)
Epoch 82 | time in 0 minutes, 5 seconds | lr=0.000015
	Loss: 0.12237(train)	|	Acc: 95.93%(train)
	Loss: 0.01004(valid)	|	Acc: 88.44%(valid)
Epoch 83 | time in 0 minutes, 5 seconds | lr=0.000014
	Loss: 0.12076(train)	|	Acc: 96.05%(train)
	Loss: 0.01043(valid)	|	Acc: 88.48%(valid)
Epoch 84 | time in 0 minutes, 5 seconds | lr=0.000013
	Loss: 0.12264(train)	|	Acc: 96.02%(train)
	Loss: 0.01070(valid)	|	Acc: 88.46%(valid)
Epoch 85 | time in 0 minutes, 5 seconds | lr=0.000013
	Loss: 0.12289(train)	|	Acc: 95.96%(train)
	Loss: 0.01159(valid)	|	Acc: 88.46%(valid)
Epoch 86 | time in 0 minutes, 5 seconds | lr=0.000012
	Loss: 0.12097(train)	|	Acc: 96.10%(train)
	Loss: 0.00764(valid)	|	Acc: 88.48%(valid)
Epoch 87 | time in 0

### early stopping
* validation setでのaccuracyが4回連続で最高値を下回ったら訓練を終えることにする。
* early stoppingの実現については、PyTorch Lightningを使う手もある。
 * https://pytorch-lightning.readthedocs.io/en/latest/early_stopping.html

In [46]:
model = EmbedTextSentiment(EMBED_DIM, NUM_CLASS, INPUT_DIM, padding_idx=PAD_IDX, p=0.5).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.95)

In [47]:
patience = 4
early_stop_count = 0
best_valid_acc = 0.0

MIN_N_EPOCHS = 10 # 最低このエポック数は実行する
N_EPOCHS = 100 # エポック数を増やしておく
for epoch in range(N_EPOCHS):

  start_time = time.time()
  train_loss, train_acc = train(train_iterator, model, optimizer, scheduler, criterion)
  valid_loss, valid_acc = test(valid_iterator, model, criterion)

  secs = int(time.time() - start_time)
  mins = secs // 60
  secs = secs % 60

  print(f'Epoch {epoch + 1} | time in {mins:d} minutes, {secs:d} seconds | ', end='')
  for param_group in optimizer.param_groups:
    print(f'lr={param_group["lr"]:.6f}')
    break
  print(f'\tLoss: {train_loss:.5f}(train)\t|\tAcc: {train_acc * 100:.2f}%(train)')
  print(f'\tLoss: {valid_loss:.5f}(valid)\t|\tAcc: {valid_acc * 100:.2f}%(valid)')

  # early stopping
  if epoch + 1 > MIN_N_EPOCHS:
    if best_valid_acc <= valid_acc:
      best_valid_acc = valid_acc
      early_stop_count = 0
    else:
      early_stop_count += 1
      if early_stop_count == patience:
        break

Epoch 1 | time in 0 minutes, 5 seconds | lr=0.000950
	Loss: 0.59996(train)	|	Acc: 67.92%(train)
	Loss: 0.02367(valid)	|	Acc: 71.64%(valid)
Epoch 2 | time in 0 minutes, 5 seconds | lr=0.000902
	Loss: 0.55894(train)	|	Acc: 71.13%(train)
	Loss: 0.02175(valid)	|	Acc: 74.22%(valid)
Epoch 3 | time in 0 minutes, 5 seconds | lr=0.000857
	Loss: 0.54009(train)	|	Acc: 72.75%(train)
	Loss: 0.02124(valid)	|	Acc: 76.92%(valid)
Epoch 4 | time in 0 minutes, 5 seconds | lr=0.000815
	Loss: 0.52548(train)	|	Acc: 74.31%(train)
	Loss: 0.01911(valid)	|	Acc: 76.92%(valid)
Epoch 5 | time in 0 minutes, 5 seconds | lr=0.000774
	Loss: 0.51180(train)	|	Acc: 75.15%(train)
	Loss: 0.01855(valid)	|	Acc: 78.24%(valid)
Epoch 6 | time in 0 minutes, 5 seconds | lr=0.000735
	Loss: 0.49469(train)	|	Acc: 76.28%(train)
	Loss: 0.01846(valid)	|	Acc: 79.76%(valid)
Epoch 7 | time in 0 minutes, 5 seconds | lr=0.000698
	Loss: 0.48739(train)	|	Acc: 76.65%(train)
	Loss: 0.01656(valid)	|	Acc: 78.56%(valid)
Epoch 8 | time in 0 minutes

## 07-05 テストセット上で評価
* 見つけ出したベストな設定を使って、テストセット上での最終的な評価をおこなう。

In [48]:
print('Checking the results of test dataset...')
test_loss, test_acc = test(test_iterator, model, criterion)
print(f'\tLoss: {test_loss:.5f}(test)\t|\tAcc: {test_acc * 100:.2f}%(test)')

Checking the results of test dataset...
	Loss: 0.00310(test)	|	Acc: 88.01%(test)


# MLPだけを使って（RNNを使わずに）単語列データの感情分析（ポジネガの二値分類）を行うコードを、PyTorchで書いてください。


# 一つ前の課題との違いは、単語埋め込みも今回は同時に一から学習する、という点です。（一つ前の課題では、fastTextの単語ベクトルを定数としてそのまま使っていました。）



